In [255]:
# fill the area id column
import os 
import pandas as pd

def fill_area_code(df):

    df['AreaID'].dropna(inplace=True)
    cn_id = df['AreaID'][0]
    print(cn_id)
    df['AreaID'] = cn_id
    
    return df
        

In [256]:
import pandas as pd
import os

def read_and_concatenate(folder_path):
    # Lists to store DataFrames
    gen_dataframes = []
    load_dataframes = []

    # Iterate over all files in the folder
    list_files = [file for file in os.listdir(folder_path) if file!='test.csv']
    for file in list_files:
        if file.endswith('.csv'):
            print('---------------------------------')
            print(file)
            file_path = os.path.join(folder_path, file)
            

            # Read 'gen' files
            if file.startswith('gen'):
                df = pd.read_csv(file_path)
                if file == 'gen_SP_B10.csv':
                    df['AreaID'] = '10YES-REE------0'
                df = fill_area_code(df)
                df['quantity'].fillna(0, inplace=True)
                gen_dataframes.append(df)


            # Read 'load' files
            elif file.startswith('load'):
                df = pd.read_csv(file_path)
                df = fill_area_code(df)
                df['Load'].fillna(0, inplace=True)
                load_dataframes.append(df)

           
    # Concatenate DataFrames vertically
    gen_concatenated = pd.concat(gen_dataframes, axis=0, ignore_index=True)
    load_concatenated = pd.concat(load_dataframes, axis=0, ignore_index=True)

    # Combine 'gen' and 'load' DataFrames
    combined_dataframe = pd.concat([gen_concatenated, load_concatenated], axis=0, ignore_index=True)

    return combined_dataframe

# Provide the path to your folder containing the CSV files
folder_path = '../data/raw_data/'
data = read_and_concatenate(folder_path)

# Optionally, save the combined DataFrame to a new CSV file
#combined_df.to_csv('combined_dataset.csv', index=False)


---------------------------------
gen_DE_B01.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B09.csv


10Y1001A1001A83F
---------------------------------
gen_DE_B10.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B11.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B12.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B15.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B16.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B18.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B19.csv
10Y1001A1001A83F
---------------------------------
gen_DK_B01.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B16.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B18.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B19.csv
10Y1001A1001A65H
---------------------------------
gen_HU_B01.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B09.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B11.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B12.csv
10YHU-MAVI

In [257]:
data['AreaID'].unique()

array(['10Y1001A1001A83F', '10Y1001A1001A65H', '10YHU-MAVIR----U',
       '10YIT-GRTN-----B', '10YNL----------L', '10YPL-AREA-----S',
       '10YSE-1--------K', '10YES-REE------0'], dtype=object)

In [258]:
data['StartTime'] = pd.to_datetime(data['StartTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['EndTime'] = pd.to_datetime(data['EndTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])

In [259]:
data['AreaID'] = data['AreaID'].replace({
    '10YHU-MAVIR----U': 'HU',
    '10YIT-GRTN-----B': 'IT',
    '10YPL-AREA-----S': 'PO',
    '10YES-REE------0': 'SP',
    '10Y1001A1001A92E': 'UK',
    '10Y1001A1001A83F': 'DE',
    '10Y1001A1001A65H': 'DK',
    '10YSE-1--------K': 'SE',
    '10YNL----------L': 'NE'
})

In [260]:
data['AreaID'].unique()

array(['DE', 'DK', 'HU', 'IT', 'NE', 'PO', 'SE', 'SP'], dtype=object)

In [261]:
data.fillna(0, inplace=True)
data['gen/load']='load'
data['Load'] = data['Load'].fillna(0)
data.loc[data['Load']==0,'gen/load']='gen'

data['power']=data['quantity']+data['Load']
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0
...,...,...,...,...,...,...,...,...,...
1623568,2023-01-01 22:30:00,2023-01-01 22:45:00,SP,MAW,0,0.0,20716.0,load,20716.0
1623569,2023-01-01 22:45:00,2023-01-01 23:00:00,SP,MAW,0,0.0,20588.0,load,20588.0
1623570,2023-01-01 23:00:00,2023-01-01 23:15:00,SP,MAW,0,0.0,20116.0,load,20116.0
1623571,2023-01-01 23:15:00,2023-01-01 23:30:00,SP,MAW,0,0.0,19708.0,load,19708.0


In [262]:
# Extract date and hour
data['Date'] = data['StartTime'].dt.date
data['Hour'] = data['StartTime'].dt.hour
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power,Date,Hour
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0,2021-12-31,23
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0,2022-01-01,0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0,2022-01-01,0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0,2022-01-01,0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0,2022-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...
1623568,2023-01-01 22:30:00,2023-01-01 22:45:00,SP,MAW,0,0.0,20716.0,load,20716.0,2023-01-01,22
1623569,2023-01-01 22:45:00,2023-01-01 23:00:00,SP,MAW,0,0.0,20588.0,load,20588.0,2023-01-01,22
1623570,2023-01-01 23:00:00,2023-01-01 23:15:00,SP,MAW,0,0.0,20116.0,load,20116.0,2023-01-01,23
1623571,2023-01-01 23:15:00,2023-01-01 23:30:00,SP,MAW,0,0.0,19708.0,load,19708.0,2023-01-01,23


In [263]:
aggregated_data = data.groupby(['AreaID', 'gen/load', 'Date', 'Hour'])['power'].sum().reset_index()

aggregated_data['concatenated'] = aggregated_data['AreaID']  + aggregated_data['gen/load']

In [264]:
pivot = aggregated_data.pivot_table(
index=['Date', 'Hour'],
columns=['concatenated'],
values='power',
aggfunc='sum'
)
pivot

concatenated        DEgen    DEload   DKgen  DKload   HUgen   HUload   ITgen  \
Date       Hour                                                                
2021-12-31 23     39401.0   42196.0  3652.0  3314.0   285.0   4254.0  5801.0   
2022-01-01 0     157373.0  165125.0  3605.0  3218.0  1376.0  16457.0  5745.0   
           1     151747.0  160415.0  3309.0  3126.0  1526.0  15426.0  6228.0   
           2     147938.0  158035.0  3043.0  3080.0  1560.0  14781.0  6407.0   
           3     141816.0  157016.0  2852.0  3044.0  1287.0  14630.0  5760.0   
...                   ...       ...     ...     ...     ...      ...     ...   
2023-01-01 19    133596.0  177225.0  4168.0  3507.0   667.0  17873.0  3873.0   
           20    131681.0  172484.0  3975.0  3354.0   648.0  17141.0  3690.0   
           21    122205.0  166627.0  3498.0  3229.0   658.0  16259.0  3460.0   
           22    120116.0  153785.0  2879.0  3055.0   623.0  15273.0  3283.0   
           23     92697.0  110140.0     NaN     NaN   468.0  10900.0     NaN   

concatenated      ITload    NEgen   NEload   POgen   POload    SEgen   SEload  \
Date       Hour                                                                 
2021-12-31 23    21121.0   3554.0  10326.0  4641.0  14438.0  11060.0  15331.0   
2022-01-01 0     19756.0  12768.0  40706.0  4491.0  13935.0  11107.0  15331.0   
           1     18685.0  11131.0  39465.0  4436.0  13579.0  11036.0  15270.0   
           2     18124.0  11324.0  38923.0  4568.0  13397.0  10509.0  15150.0   
           3     18400.0  11290.0  38211.0  4559.0  13364.0  10770.0  15387.0   
...                  ...      ...      ...     ...      ...      ...      ...   
2023-01-01 19    24617.0   8730.0  47635.0  4968.0  15468.0  12347.0  16145.0   
           20    22944.0   7654.0  45689.0  4220.0  14698.0  11986.0  15869.0   
           21    20791.0   6577.0  43276.0  3525.0  13749.0  11227.0  15143.0   
           22    19198.0   7759.0  41269.0  3236.0  13061.0  10515.0  14697.0   
           23        NaN   5441.0  29948.0     NaN      NaN      NaN      NaN   

concatenated       SPgen   SPload  
Date       Hour                    
2021-12-31 23    11244.0  20827.0  
2022-01-01 0     10827.0  19530.0  
           1     11140.0  18383.0  
           2     11361.0  17680.0  
           3     11247.0  17396.0  
...                  ...      ...  
2023-01-01 19    76316.0  98224.0  
           20    76276.0  97620.0  
           21    72280.0  91896.0  
           22    65968.0  84324.0  
           23    49836.0  59140.0  

[8785 rows x 16 columns]

In [265]:
import numpy as np
# Replace zeros with NaN
pivot.replace(0, np.nan, inplace=True)

# Drop rows where all elements are NaN
pivot=pivot.dropna()



In [266]:
pivot=pivot.reset_index()
pivot['Date'] = pd.to_datetime(pivot['Date'])
pivot = pivot[pivot['Date'].dt.year == 2022]
pivot

concatenated,Date,Hour,DEgen,DEload,DKgen,DKload,HUgen,HUload,ITgen,ITload,NEgen,NEload,POgen,POload,SEgen,SEload,SPgen,SPload
1,2022-01-01,0,157373.0,165125.0,3605.0,3218.0,1376.0,16457.0,5745.0,19756.0,12768.0,40706.0,4491.0,13935.0,11107.0,15331.0,10827.0,19530.0
2,2022-01-01,1,151747.0,160415.0,3309.0,3126.0,1526.0,15426.0,6228.0,18685.0,11131.0,39465.0,4436.0,13579.0,11036.0,15270.0,11140.0,18383.0
3,2022-01-01,2,147938.0,158035.0,3043.0,3080.0,1560.0,14781.0,6407.0,18124.0,11324.0,38923.0,4568.0,13397.0,10509.0,15150.0,11361.0,17680.0
4,2022-01-01,3,141816.0,157016.0,2852.0,3044.0,1287.0,14630.0,5760.0,18400.0,11290.0,38211.0,4559.0,13364.0,10770.0,15387.0,11247.0,17396.0
5,2022-01-01,4,131970.0,154552.0,2724.0,3130.0,1309.0,14688.0,5501.0,19223.0,11093.0,38146.0,4374.0,13449.0,10852.0,15737.0,10868.0,17544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,175631.0,184106.0,1765.0,3713.0,851.0,17488.0,4198.0,24809.0,17241.0,45646.0,6959.0,15211.0,10179.0,14440.0,67056.0,104484.0
8756,2022-12-31,20,176499.0,175401.0,1593.0,3579.0,888.0,17075.0,3676.0,23169.0,17233.0,43425.0,6910.0,14641.0,9586.0,14201.0,62432.0,97124.0
8757,2022-12-31,21,175678.0,168830.0,1788.0,3520.0,807.0,16717.0,3641.0,21857.0,17129.0,41766.0,7054.0,13977.0,9148.0,13713.0,61096.0,87316.0
8758,2022-12-31,22,165104.0,161541.0,2242.0,3440.0,754.0,16261.0,3422.0,20555.0,15939.0,40415.0,5946.0,13272.0,8806.0,13772.0,55252.0,82064.0


In [267]:
pivot.dropna()

concatenated,Date,Hour,DEgen,DEload,DKgen,DKload,HUgen,HUload,ITgen,ITload,NEgen,NEload,POgen,POload,SEgen,SEload,SPgen,SPload
1,2022-01-01,0,157373.0,165125.0,3605.0,3218.0,1376.0,16457.0,5745.0,19756.0,12768.0,40706.0,4491.0,13935.0,11107.0,15331.0,10827.0,19530.0
2,2022-01-01,1,151747.0,160415.0,3309.0,3126.0,1526.0,15426.0,6228.0,18685.0,11131.0,39465.0,4436.0,13579.0,11036.0,15270.0,11140.0,18383.0
3,2022-01-01,2,147938.0,158035.0,3043.0,3080.0,1560.0,14781.0,6407.0,18124.0,11324.0,38923.0,4568.0,13397.0,10509.0,15150.0,11361.0,17680.0
4,2022-01-01,3,141816.0,157016.0,2852.0,3044.0,1287.0,14630.0,5760.0,18400.0,11290.0,38211.0,4559.0,13364.0,10770.0,15387.0,11247.0,17396.0
5,2022-01-01,4,131970.0,154552.0,2724.0,3130.0,1309.0,14688.0,5501.0,19223.0,11093.0,38146.0,4374.0,13449.0,10852.0,15737.0,10868.0,17544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,175631.0,184106.0,1765.0,3713.0,851.0,17488.0,4198.0,24809.0,17241.0,45646.0,6959.0,15211.0,10179.0,14440.0,67056.0,104484.0
8756,2022-12-31,20,176499.0,175401.0,1593.0,3579.0,888.0,17075.0,3676.0,23169.0,17233.0,43425.0,6910.0,14641.0,9586.0,14201.0,62432.0,97124.0
8757,2022-12-31,21,175678.0,168830.0,1788.0,3520.0,807.0,16717.0,3641.0,21857.0,17129.0,41766.0,7054.0,13977.0,9148.0,13713.0,61096.0,87316.0
8758,2022-12-31,22,165104.0,161541.0,2242.0,3440.0,754.0,16261.0,3422.0,20555.0,15939.0,40415.0,5946.0,13272.0,8806.0,13772.0,55252.0,82064.0


In [268]:
for country in ['HU','IT','PO','SP','DE','DK','SE','NE']:
    pivot[country+'_surplus']=pivot[country+'gen']-pivot[country+'load']


In [269]:
pivot

concatenated,Date,Hour,DEgen,DEload,DKgen,DKload,HUgen,HUload,ITgen,ITload,...,SPgen,SPload,HU_surplus,IT_surplus,PO_surplus,SP_surplus,DE_surplus,DK_surplus,SE_surplus,NE_surplus
1,2022-01-01,0,157373.0,165125.0,3605.0,3218.0,1376.0,16457.0,5745.0,19756.0,...,10827.0,19530.0,-15081.0,-14011.0,-9444.0,-8703.0,-7752.0,387.0,-4224.0,-27938.0
2,2022-01-01,1,151747.0,160415.0,3309.0,3126.0,1526.0,15426.0,6228.0,18685.0,...,11140.0,18383.0,-13900.0,-12457.0,-9143.0,-7243.0,-8668.0,183.0,-4234.0,-28334.0
3,2022-01-01,2,147938.0,158035.0,3043.0,3080.0,1560.0,14781.0,6407.0,18124.0,...,11361.0,17680.0,-13221.0,-11717.0,-8829.0,-6319.0,-10097.0,-37.0,-4641.0,-27599.0
4,2022-01-01,3,141816.0,157016.0,2852.0,3044.0,1287.0,14630.0,5760.0,18400.0,...,11247.0,17396.0,-13343.0,-12640.0,-8805.0,-6149.0,-15200.0,-192.0,-4617.0,-26921.0
5,2022-01-01,4,131970.0,154552.0,2724.0,3130.0,1309.0,14688.0,5501.0,19223.0,...,10868.0,17544.0,-13379.0,-13722.0,-9075.0,-6676.0,-22582.0,-406.0,-4885.0,-27053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,175631.0,184106.0,1765.0,3713.0,851.0,17488.0,4198.0,24809.0,...,67056.0,104484.0,-16637.0,-20611.0,-8252.0,-37428.0,-8475.0,-1948.0,-4261.0,-28405.0
8756,2022-12-31,20,176499.0,175401.0,1593.0,3579.0,888.0,17075.0,3676.0,23169.0,...,62432.0,97124.0,-16187.0,-19493.0,-7731.0,-34692.0,1098.0,-1986.0,-4615.0,-26192.0
8757,2022-12-31,21,175678.0,168830.0,1788.0,3520.0,807.0,16717.0,3641.0,21857.0,...,61096.0,87316.0,-15910.0,-18216.0,-6923.0,-26220.0,6848.0,-1732.0,-4565.0,-24637.0
8758,2022-12-31,22,165104.0,161541.0,2242.0,3440.0,754.0,16261.0,3422.0,20555.0,...,55252.0,82064.0,-15507.0,-17133.0,-7326.0,-26812.0,3563.0,-1198.0,-4966.0,-24476.0


In [270]:
import pandas as pd

# Assuming 'Date' is in datetime format, if not, convert it
pivot['Date'] = pd.to_datetime(pivot['Date'])

# Define European dates for seasons
spring_start = pd.to_datetime('2023-03-21')
summer_start = pd.to_datetime('2023-06-21')
autumn_start = pd.to_datetime('2023-09-22')
winter_start = pd.to_datetime('2023-12-21')

# Explicitly define bin edges and labels
bin_edges = [pd.Timestamp.min, spring_start, summer_start, autumn_start, winter_start, pd.Timestamp.max]
labels = ['min_to_spring', 'spring', 'summer', 'autumn', 'winter']

# Create a new column 'season' based on the defined seasons
pivot['season'] = pd.cut(
    pivot['Date'],
    bins=bin_edges,
    labels=labels,
    right=False
)

pivot['season'] = pivot['season'].replace('min_to_spring', 'winter')

# One-hot encode the 'season' column
pivot = pd.get_dummies(pivot, columns=['season'], drop_first=True)

# Extract day of the week and create 'weekend' column
pivot['day_of_week'] = pivot['Date'].dt.dayofweek
pivot['is_weekend'] = pd.get_dummies(pivot['day_of_week'].isin([5, 6]).astype(int), drop_first=True)

In [271]:
# Map column names to numbers
column_to_number = {
    'HU_surplus': 0,
    'IT_surplus': 1,
    'PO_surplus': 2,
    'SP_surplus': 3,
    'DE_surplus': 4,
    'DK_surplus': 5,
    'SE_surplus': 6,
    'NE_surplus': 7
}


# Find the column with the maximum value for each row and map it to the corresponding number
pivot['label'] = pivot[['HU_surplus', 'IT_surplus', 'PO_surplus', 'SP_surplus', 
                  'DE_surplus', 'DK_surplus', 'SE_surplus', 'NE_surplus']].idxmax(axis=1).map(column_to_number)




In [272]:
pivot.to_csv('../data/final_data.csv',index=False)

In [273]:
pivot

,Date,Hour,DEgen,DEload,DKgen,DKload,HUgen,HUload,ITgen,ITload,...,DE_surplus,DK_surplus,SE_surplus,NE_surplus,season_summer,season_autumn,season_winter,day_of_week,is_weekend,label
1,2022-01-01,0,157373.0,165125.0,3605.0,3218.0,1376.0,16457.0,5745.0,19756.0,...,-7752.0,387.0,-4224.0,-27938.0,False,False,True,5,True,5
2,2022-01-01,1,151747.0,160415.0,3309.0,3126.0,1526.0,15426.0,6228.0,18685.0,...,-8668.0,183.0,-4234.0,-28334.0,False,False,True,5,True,5
3,2022-01-01,2,147938.0,158035.0,3043.0,3080.0,1560.0,14781.0,6407.0,18124.0,...,-10097.0,-37.0,-4641.0,-27599.0,False,False,True,5,True,5
4,2022-01-01,3,141816.0,157016.0,2852.0,3044.0,1287.0,14630.0,5760.0,18400.0,...,-15200.0,-192.0,-4617.0,-26921.0,False,False,True,5,True,5
5,2022-01-01,4,131970.0,154552.0,2724.0,3130.0,1309.0,14688.0,5501.0,19223.0,...,-22582.0,-406.0,-4885.0,-27053.0,False,False,True,5,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,175631.0,184106.0,1765.0,3713.0,851.0,17488.0,4198.0,24809.0,...,-8475.0,-1948.0,-4261.0,-28405.0,False,False,True,5,True,5
8756,2022-12-31,20,176499.0,175401.0,1593.0,3579.0,888.0,17075.0,3676.0,23169.0,...,1098.0,-1986.0,-4615.0,-26192.0,False,False,True,5,True,4
8757,2022-12-31,21,175678.0,168830.0,1788.0,3520.0,807.0,16717.0,3641.0,21857.0,...,6848.0,-1732.0,-4565.0,-24637.0,False,False,True,5,True,4
8758,2022-12-31,22,165104.0,161541.0,2242.0,3440.0,754.0,16261.0,3422.0,20555.0,...,3563.0,-1198.0,-4966.0,-24476.0,False,False,True,5,True,4
